# Data Preprocessing

In [1]:
!git clone https://github.com/indichealth/indic-health-demo.git

Cloning into 'indic-health-demo'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 231 (delta 55), reused 103 (delta 43), pack-reused 110
Receiving objects: 100% (231/231), 1.24 MiB | 1.22 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [2]:
%cd indic-health-demo/Dataset

/content/indic-health-demo/Dataset


In [3]:
import pandas as pd
import os

In [4]:
data_path = 'IHQID-WebMD'

In [5]:
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_path, 'test.csv'))

In [6]:
src = 'bengali'
bridge = 'hindi'
tar = 'english'

In [7]:
train_df = train_df.drop(columns=f'question_{bridge}')
test_df = test_df.drop(columns=f'question_{bridge}')
train_df = train_df.drop(columns=f'question_{tar}')
test_df = test_df.drop(columns=f'question_{tar}')

In [8]:
train_df['question_bengali'][0]

'নিস্টাটিন কি জন্য নির্ধারিত হয়?'

In [9]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 902.5 kB/s eta 0:00:00


In [10]:
from deep_translator import GoogleTranslator

def translate_queries(data, src, tar):
  def translate(sentence, src, tar):
    return GoogleTranslator(source=src, target=tar).translate(sentence)

  s, t = src[:2], tar[:2]
  data[f'question_{tar}'] = data[f'question_{src}'].apply(lambda x: translate(x, s, t))
  return data

# translation from source to bridge and then to target language
train_ = translate_queries(train_df, src, bridge).drop(columns=[f'question_{src}'])
test_ = translate_queries(test_df, src, bridge).drop(columns=[f'question_{src}'])

train = translate_queries(train_, bridge, tar).drop(columns=[f'question_{bridge}'])
test = translate_queries(test_, bridge, tar).drop(columns=[f'question_{bridge}'])

In [11]:
train['question_english']

0                       What is nystatin prescribed for?
1      Can washing hands after sex prevent me from ge...
2                       Does Percocet cause weight gain?
3         Can a glass of wine cause high blood pressure?
4                      Is too much buttermilk the cause?
                             ...                        
715    Can a dependent require the insurance company ...
716         How can I use duct tape to get rid of warts?
717    Can facial exercises be done to recover from f...
718                         Is prenatal ultrasound safe?
719    How can I reduce inguinal hernia symptoms unti...
Name: question_english, Length: 720, dtype: object

# Model Preparation

In [12]:
!pip install transformers[torch]
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [13]:
from transformers import AutoTokenizer
model_id = 'emilyalsentzer/Bio_ClinicalBERT'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [14]:
def biotagging(data: pd.DataFrame, lang):
  data = data.fillna('') # replace nan values with empty string
  # add column for tokens from tokenizer
  def tokens(text):
    encoded_input = tokenizer.encode(text, return_tensors='pt')
    decoded_input = tokenizer.convert_ids_to_tokens(encoded_input[0])
    decoded_input = [token.lower() for token in decoded_input]
    return decoded_input

  cols = [f'question_{lang}',
          f'disease_{lang}',
          f'drug_{lang}',
          f'treatment_{lang}',]
  for col in cols:
    data[col + '_tokens'] = data[col].apply(tokens)

  def biotag(row):
    qn_tokens = row[f'question_{lang}_tokens'][1:-1]
    dis_tokens = row[f'disease_{lang}_tokens'][1:-1]      # for disease, drug and treatment remove <s> and </s> tokens for proper matching (NOTE : for empty string, this gives empty list)
    drug_tokens = row[f'drug_{lang}_tokens'][1:-1]
    treat_tokens = row[f'treatment_{lang}_tokens'][1:-1]

    i = 0
    biotags = []
    while i < len(qn_tokens):
      token = qn_tokens[i]
      # if current token matches with the starting tokens of annotated disease, drug or treatment
      if ((len(dis_tokens) > 0 and (token in dis_tokens[0] or dis_tokens[0] in token)) or
         (len(drug_tokens) > 0 and (token in drug_tokens[0] or drug_tokens[0] in token)) or
         (len(treat_tokens) > 0 and (token in treat_tokens[0] or treat_tokens[0] in token))):

        entity = ""   # label for detected entity
        if len(dis_tokens) > 0 and (token in dis_tokens[0] or dis_tokens[0] in token):
          entity_tokens = dis_tokens
          entity = "disease"
        elif len(drug_tokens) > 0 and (token in drug_tokens[0] or drug_tokens[0] in token):
          entity_tokens = drug_tokens
          entity = "drug"
        elif len(treat_tokens) > 0 and (token in treat_tokens[0] or treat_tokens[0] in token):
          entity_tokens = treat_tokens
          entity = "treatment"

        # define matching function to compute similarity of entity and question subpart's tokens
        def match_tokens(ent_toks, qn_toks, thresh):
          cnt = 0
          match_toks = []   # store the set of question tokens which match with entity tokens
          for ent_tok in ent_toks:
            for qn_tok in qn_toks:
              if (ent_tok in qn_tok) or (qn_tok in ent_tok):
                cnt += 1
                match_toks.append(qn_tok)
                break

          match_toks = list(set(match_toks))
          return cnt/len(ent_toks), match_toks

        # if detected entity approximately matches with current substring based on threshold value
        thresh = 0.8
        # print(entity_tokens)
        # print(qn_tokens[i:min(i+len(entity_tokens), len(qn_tokens))])
        f_match, match_toks = match_tokens(entity_tokens, qn_tokens[i:min(i+len(entity_tokens), len(qn_tokens))], thresh)

        # print("Match ", f_match)
        # print("Match toks ", match_toks)
        if f_match >= thresh:
          idx = i   # store current value of idx
          # add B, I tags for all tokens matching with those of the detected entity
          while i < min(idx + len(entity_tokens), len(qn_tokens)):
            if qn_tokens[i] in match_toks:
              if i == idx:
                biotags.append(f'B-{entity}')
              else:
                biotags.append(f'I-{entity}')
            else:
              biotags.append('O')
            i += 1
          continue  # to prevent skipping an extra index
        else: # since entity thought to match doesn't actually match completely, it is not the start token of some entity. So, add O biotag to the current token only
          biotags.append('O')

      else:
        biotags.append('O')
      i+=1

    # The above approach might cause some intermediate tokens for any entity to be labelled O, even though its surrounding tokens are B/I tokens. We fix such tokens as postprocessing
    i = 0
    while i < len(biotags):
      biotag = biotags[i]
      if (i>0 and i<len(biotags)-1):
        prev_tag = biotags[i-1]
        next_tag = biotags[i+1]
        if (('B-' in prev_tag or 'I-' in prev_tag) and 'I-' in next_tag):
          entity = prev_tag[2:]
          biotags[i] = 'I-' + entity
      i+=1
    return biotags

  data[f'question_{lang}_biotags'] = data.apply(biotag, axis=1)
  return data

In [15]:
x = biotagging(train.iloc[[18]], 'english')
print("Question : ", x.iloc[0][['question_english']].values[0])
print("Disease : ", x.iloc[0][['disease_english']].values[0])
print("Drug : ", x.iloc[0][['drug_english']].values[0])
print("Treatment : ", x.iloc[0][['treatment_english']].values[0])
print(x.iloc[0][['question_english_biotags']].values[0])

Question :  Scoliosis. Does it affect the stomach and breathing?
Disease :  Scoliosis
Drug :  
Treatment :  
['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [16]:
train['question_english']

0                       What is nystatin prescribed for?
1      Can washing hands after sex prevent me from ge...
2                       Does Percocet cause weight gain?
3         Can a glass of wine cause high blood pressure?
4                      Is too much buttermilk the cause?
                             ...                        
715    Can a dependent require the insurance company ...
716         How can I use duct tape to get rid of warts?
717    Can facial exercises be done to recover from f...
718                         Is prenatal ultrasound safe?
719    How can I reduce inguinal hernia symptoms unti...
Name: question_english, Length: 720, dtype: object

In [17]:
train = biotagging(train, 'english')
test = biotagging(test, 'english')

In [18]:
from datasets import Dataset

tag2label = {
    'O': 0,
    'B-disease':1,
    'I-disease':2,
    'B-drug':3,
    'I-drug':4,
    'B-treatment':5,
    'I-treatment':6
}

def preprocess(data: pd.DataFrame, lang):
  sents = list(data[f'question_{lang}'])
  labels = list(data[f'question_{lang}_biotags'].apply(lambda x:[tag2label[tag] for tag in x]))

  dataset = Dataset.from_dict(
      {
          'sentence': sents,
          'labels': labels
      }
  )

  max_length = 512
  special_token_label = -100    # define label for special token
  def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=max_length)

  def align_labels(examples):
    labels = examples['labels']
    for sent_labels in labels:
      sent_labels.extend([special_token_label] * (max_length - len(sent_labels)))   # add special token labels at end
    return examples

  dataset = dataset.map(tokenize_function, batched=True)
  dataset = dataset.map(align_labels, batched=True)
  return dataset

tokenized_train = preprocess(train, 'english')
tokenized_test = preprocess(test, 'english')

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

In [19]:
tokenized_train['labels'][0]

[0,
 0,
 3,
 4,
 4,
 0,
 0,
 0,
 0,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100

# Model Creation

In [20]:
from transformers import AutoModelForTokenClassification

model_name = "emilyalsentzer/Bio_ClinicalBERT"
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels = len(tag2label) + 1)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import DataCollatorForTokenClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="ner_model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=500,
    num_train_epochs=10,
    load_best_model_at_end=True,
    learning_rate=1e-5,
    push_to_hub=False,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
)

In [22]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=450, training_loss=0.3902230156792535, metrics={'train_runtime': 612.5439, 'train_samples_per_second': 11.754, 'train_steps_per_second': 0.735, 'total_flos': 1881438702796800.0, 'train_loss': 0.3902230156792535, 'epoch': 10.0})

In [23]:
# Evaluate the model on the test dataset
results = trainer.evaluate()

# Print the evaluation results
print(results)

{'eval_loss': 0.46331870555877686, 'eval_runtime': 8.0096, 'eval_samples_per_second': 30.089, 'eval_steps_per_second': 7.616, 'epoch': 10.0}


In [24]:
predictions = trainer.predict(tokenized_test)

In [25]:
pred_vals = predictions.predictions

In [26]:
pred_vals[0].shape

(512, 8)

In [27]:
import numpy as np
predicted_labels = np.argmax(pred_vals, axis=2)
predicted_labels.shape

(241, 512)

In [28]:
predicted_labels[3]

array([0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4,
       0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 4, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0,

In [29]:
tokenized_test['labels'][3]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,


In [30]:
len(tokenized_test['labels'][0])

512

In [31]:
from sklearn.metrics import classification_report, confusion_matrix

pred_labels = predicted_labels.flatten()
actual_labels = np.array(tokenized_test['labels']).flatten()

# remove special tokens before computing token classification accuracy
special_token_label = -100
pred_filtered_labels = []
actual_filtered_labels = []
for pred_label, actual_label in zip(pred_labels, actual_labels):
  if actual_label == special_token_label:
    continue
  pred_filtered_labels.append(pred_label)
  actual_filtered_labels.append(actual_label)

print(classification_report(actual_filtered_labels, pred_filtered_labels))
print(confusion_matrix(actual_filtered_labels, pred_filtered_labels))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      2778
           1       0.54      0.51      0.52       103
           2       0.54      0.67      0.60       161
           3       0.50      0.56      0.53        52
           4       0.51      0.58      0.54       142
           5       0.57      0.62      0.59        39
           6       0.60      0.64      0.62        88

    accuracy                           0.85      3363
   macro avg       0.60      0.64      0.62      3363
weighted avg       0.86      0.85      0.86      3363

[[2516   37   82   23   74   12   34]
 [  44   53    2    2    0    2    0]
 [  42    6  108    0    2    2    1]
 [  19    2    0   29    1    1    0]
 [  54    0    2    2   82    0    2]
 [  13    0    0    2    0   24    0]
 [  24    1    5    0    1    1   56]]


In [32]:
tag2label

{'O': 0,
 'B-disease': 1,
 'I-disease': 2,
 'B-drug': 3,
 'I-drug': 4,
 'B-treatment': 5,
 'I-treatment': 6}